Сначала осуществим импорт необходимых библиотек и подготовку данных

In [1]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [4]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [5]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

In [6]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [7]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [8]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-7-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 24 s, sys: 781 ms, total: 24.8 s
Wall time: 24.8 s


In [9]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 2min 54s, sys: 157 ms, total: 2min 54s
Wall time: 2min 54s


In [10]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [11]:
%%time
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)

CPU times: user 1min 12s, sys: 1min 59s, total: 3min 12s
Wall time: 39.6 s


In [12]:
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(4, 0.104512), (20, 0.21361734), (22, 0.65855604)]

In [13]:
x = lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: россия nn российский система первый новый проект
topic_1: ракета определение методика следствие инвестировать калинин мак
topic_2: nn тело взрыв всё научный произойти газета
topic_3: россия гражданин российский рубль закон ребёнок правительство
topic_4: тыс британский рак дом великобритания штат ставка
topic_5: снижение конкурс супруг индия польша польский армения
topic_6: квартира nn компьютерный женщина одежда негативно сближение
topic_7: миссия риск планета показатель население доход журнал
topic_8: оборудование высота китай озеро золото китайский место
topic_9: фонд москва пострадать банк район авария автомобиль
topic_10: статья журнал население обнаружить писать доллар день
topic_11: источник параметр парламент падать святой кризис горизонт
topic_12: погибнуть убийство операция произойти террорист пилот превысить
topic_13: военный сша газ россия млн млрд армия
topic_14: источник сотрудник смерть управление эксперимент наука врач
topic_15: остров космос лаборатория япония 

In [14]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [15]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(3)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.871018,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.053325,0.0,0.000000,0.0,0.0
1,4896,0.000000,0.0,0.752311,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.223452,0.0,0.000000,0.0,0.0
2,4897,0.000000,0.0,0.000000,0.0,0.103382,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.213660,0.0,0.659643,0.0,0.0


**Задание:**

* Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
* Повторить п.2, но используя уже не медиану, а max
* (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
* Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
* Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

Вначале посчитаем и сохраним метрики для среднего

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve


def get_scores(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall + 1e-16)
    ix = np.argmax(fscore)

    return precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds)

In [17]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [18]:
# функция немного изменена - также принимает словарь с документами
def get_user_embedding_mean(user_articles_list, docs):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([docs[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [19]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
    lambda x: get_user_embedding_mean(x, doc_dict), 1)])

user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings = pd.merge(user_embeddings, target, 'left')

X = user_embeddings.drop(['uid', 'churn'], axis=1)
y = user_embeddings['churn']

In [20]:
indices = ['precision', 'recall', 'f1', 'roc_auc']

In [21]:
score_mean = pd.DataFrame(get_scores(X, y), index=indices, columns=['mean'])
score_mean

,mean
precision,0.688596
recall,0.640816
f1,0.663848
roc_auc,0.947916


**Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке**

In [22]:
def get_user_embedding_median(user_articles_list, docs):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([docs[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

In [23]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
    lambda x: get_user_embedding_median(x, doc_dict), 1)])

user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings = pd.merge(user_embeddings, target, 'left')

X = user_embeddings.drop(['uid', 'churn'], axis=1)
y = user_embeddings['churn']

score_median = pd.DataFrame(get_scores(X, y), index=indices, columns=['median'])
score_median

,median
precision,0.694352
recall,0.853061
f1,0.765568
roc_auc,0.973175


**Повторить п.2, но используя уже не медиану, а max**

In [24]:
def get_user_embedding_max(user_articles_list, docs):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([docs[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [25]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
    lambda x: get_user_embedding_max(x, doc_dict), 1)])

user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings = pd.merge(user_embeddings, target, 'left')

X = user_embeddings.drop(['uid', 'churn'], axis=1)
y = user_embeddings['churn']

score_max = pd.DataFrame(get_scores(X, y), index=indices, columns=['max'])
score_max

,max
precision,0.845815
recall,0.783673
f1,0.813559
roc_auc,0.980962


**(опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.**

In [26]:
# веса для тем
topic_idf = np.log(topic_matrix.shape[0]/np.sum(topic_matrix > 0))[1:].values

# словарь документов, взвешенных по темам
idf_doc_dict = dict(
    zip(topic_matrix['doc_id'].values, topic_matrix[
    ['topic_{}'.format(i) for i in range(25)]
].values * topic_idf)
)

In [27]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
    lambda x: get_user_embedding_mean(x, idf_doc_dict), 1)])

user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings = pd.merge(user_embeddings, target, 'left')

X = user_embeddings.drop(['uid', 'churn'], axis=1)
y = user_embeddings['churn']

score_idf_mean = pd.DataFrame(get_scores(X, y), index=indices, columns=['idf_mean'])
score_idf_mean

,idf_mean
precision,0.763948
recall,0.726531
f1,0.744770
roc_auc,0.965828


**Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score**

In [28]:
score = pd.concat([score_mean, score_median, score_max, score_idf_mean], axis=1)
score

,mean,median,max,idf_mean
precision,0.688596,0.694352,0.845815,0.763948
recall,0.640816,0.853061,0.783673,0.726531
f1,0.663848,0.765568,0.813559,0.744770
roc_auc,0.947916,0.973175,0.980962,0.965828


**Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных**

Наилучшим образом показал себя метод max (по precision и recall). Вероятно это связано с тем, что при усреднении мы теряем часть информации о степени заинтересованности пользователя той или иной темой. Ниже приведен пример.

Есть два пользователя:
* Один пользователь прочитал статью, которая с высокой долей вероятности (допустим 0.6) относится к теме финансов (+ еще какие-то вероятности для других тем). Затем прочитал статью, которая относится к теме спорта, но при этом у статьи низкая вероятность принадлежности к теме финансов (допустим 0.01). Итоговая заинтересованность финансами будет ~0.3 (среднее между 0.6 и 0.01), хотя пользователь читал статью, которую большая часть пользователей возможно не будет читать (из-за обилия специфичных терминов).

* Другой пользователь читает статьи на общие темы, которые алгоритм относит к теме финансов с вероятностью 0.2-0.3.

Заинтересованность финансами, при усреднении, у обоих пользователей будет примерно равна, но при этом их степень заинтересованности темой могут значительно различаться. 

Выбор max позволяет лучше описать степень заинтересованности пользователя той или иной темой (лучше сегментировать пользователей по интересам).

Перевзвешивание документов по темам позволило немного 'почистить от мусора' документы, придав малые веса часто упоминаемым темам, поэтому качество у idf_mean в целом выше, чем у mean.